## ANN model prediction

In [1]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import pandas as pd
import numpy as np


2025-12-19 04:28:03.481879: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-12-19 04:28:03.542450: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-12-19 04:28:05.115260: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
/home/hashaneranga/.virtualenvs/MachineLearning_and_datamining_fundamentals/lib/python3.12/

In [2]:
# load the trained model, scaler and onehotencoder

model = load_model('model.keras')

with open('label_encoder_geography.pkl', 'rb') as file:
    label_encoder_geo = pickle.load(file)

with open('label_encoder_gender.pkl', 'rb') as file:
    label_encoder_gender = pickle.load(file)

with open('scaler.pkl', 'rb') as file:
    scaler = pickle.load(file)

I0000 00:00:1766098838.045586   32904 gpu_device.cc:2020] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 5563 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4060 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.9
/home/hashaneranga/.virtualenvs/MachineLearning_and_datamining_fundamentals/lib/python3.12/site-packages/keras/src/saving/saving_lib.py:797: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 8 variables whereas the saved optimizer has 14 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [4]:
input_data = {
    'CreditScore': [650, 720, 580, 800],
    'Geography': ['France', 'Germany', 'Spain', 'France'],
    'Gender': ['Female', 'Male', 'Female', 'Male'],
    'Age': [35, 42, 28, 55],
    'Tenure': [5, 8, 2, 10],
    'Balance': [80000.00, 120000.50, 0.00, 150000.75],
    'NumOfProducts': [2, 1, 1, 3],
    'HasCrCard': [1, 1, 0, 1],
    'IsActiveMember': [1, 0, 1, 0],
    'EstimatedSalary': [75000.00, 95000.00, 45000.00, 120000.00]
}

input_df = pd.DataFrame(input_data)
print("Sample Input Data:")
input_df

Sample Input Data:


,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,650,France,Female,35,5,80000.00,2,1,1,75000.0
1,720,Germany,Male,42,8,120000.50,1,1,0,95000.0
2,580,Spain,Female,28,2,0.00,1,0,1,45000.0
3,800,France,Male,55,10,150000.75,3,1,0,120000.0


In [5]:
# Step 1: Encode Gender using Label Encoder
input_df['Gender'] = label_encoder_gender.transform(input_df['Gender'])

print("After Gender Encoding:")
input_df

After Gender Encoding:


,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,650,France,0,35,5,80000.00,2,1,1,75000.0
1,720,Germany,1,42,8,120000.50,1,1,0,95000.0
2,580,Spain,0,28,2,0.00,1,0,1,45000.0
3,800,France,1,55,10,150000.75,3,1,0,120000.0


In [6]:
# Step 2: Encode Geography using OneHotEncoder
geo_encoded = label_encoder_geo.transform(input_df[['Geography']])
geo_encoded_df = pd.DataFrame(
    data=geo_encoded,
    columns=label_encoder_geo.get_feature_names_out()
)

print("Geography Encoded:")
geo_encoded_df

Geography Encoded:


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,1.0,0.0
2,0.0,0.0,1.0
3,1.0,0.0,0.0


In [7]:
# Step 3: Combine encoded geography with other features
input_df = pd.concat([input_df.drop(['Geography'], axis=1), geo_encoded_df], axis=1)

print("After Geography Encoding - Final Features:")
input_df

After Geography Encoding - Final Features:


,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,650,0,35,5,80000.00,2,1,1,75000.0,1.0,0.0,0.0
1,720,1,42,8,120000.50,1,1,0,95000.0,0.0,1.0,0.0
2,580,0,28,2,0.00,1,0,1,45000.0,0.0,0.0,1.0
3,800,1,55,10,150000.75,3,1,0,120000.0,1.0,0.0,0.0


In [8]:
# Step 4: Scale the features using the saved scaler
input_scaled = scaler.transform(input_df)

print("Input shape after preprocessing:", input_scaled.shape)
print("\nScaled Input Data:")
print(input_scaled)

Input shape after preprocessing: (4, 12)

Scaled Input Data:
[[-1.70986091e-02 -1.09499335e+00 -3.70568589e-01 -1.34471949e-03
   6.24086011e-02  8.08436154e-01  6.49202671e-01  9.74816989e-01
  -4.42165451e-01  1.00150113e+00 -5.79467227e-01 -5.76388018e-01]
 [ 7.09342564e-01  9.13247552e-01  2.94938465e-01  1.03972843e+00
   7.02856186e-01 -9.16687674e-01  6.49202671e-01 -1.02583358e+00
  -9.44320445e-02 -9.98501123e-01  1.72572313e+00 -5.76388018e-01]
 [-7.43539782e-01 -1.09499335e+00 -1.03607564e+00 -1.04241787e+00
  -1.21847056e+00 -9.16687674e-01 -1.54035103e+00  9.74816989e-01
  -9.63765560e-01 -9.98501123e-01 -5.79467227e-01  1.73494238e+00]
 [ 1.53956105e+00  9.13247552e-01  1.53088014e+00  1.73377720e+00
   1.18318987e+00  2.53355998e+00  6.49202671e-01 -1.02583358e+00
   3.40234713e-01  1.00150113e+00 -5.79467227e-01 -5.76388018e-01]]


In [9]:
# Step 5: Make predictions
predictions = model.predict(input_scaled)

print("Churn Probabilities:")
print(predictions)

2025-12-19 04:41:40.775504: I external/local_xla/xla/service/service.cc:163] XLA service 0x76dab0004830 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2025-12-19 04:41:40.775666: I external/local_xla/xla/service/service.cc:171]   StreamExecutor device (0): NVIDIA GeForce RTX 4060 Laptop GPU, Compute Capability 8.9
2025-12-19 04:41:40.811861: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2025-12-19 04:41:40.877632: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:473] Loaded cuDNN version 91700


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 737ms/step
Churn Probabilities:
[[0.06055267]
 [0.5328428 ]
 [0.2064219 ]
 [0.9993155 ]]


I0000 00:00:1766099501.423652   37788 device_compiler.h:196] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


In [10]:
# Step 6: Create results DataFrame with predictions
binary_predictions = (predictions > 0.5).astype(int)

# Recreate original input data for display
results_df = pd.DataFrame(input_data)
results_df['Churn_Probability'] = predictions.flatten()
results_df['Predicted_Churn'] = binary_predictions.flatten()
results_df['Risk_Level'] = results_df['Churn_Probability'].apply(
    lambda x: 'High' if x > 0.7 else ('Medium' if x > 0.4 else 'Low')
)

print("\nPrediction Results:")
results_df


Prediction Results:


,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Churn_Probability,Predicted_Churn,Risk_Level
0,650,France,Female,35,5,80000.00,2,1,1,75000.0,0.060553,0,Low
1,720,Germany,Male,42,8,120000.50,1,1,0,95000.0,0.532843,1,Medium
2,580,Spain,Female,28,2,0.00,1,0,1,45000.0,0.206422,0,Low
3,800,France,Male,55,10,150000.75,3,1,0,120000.0,0.999316,1,High
